In [1]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.8}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.8}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
    "ee": {
        "answers": {
            "P": 0.5,
            "P_given_Q": 0.25,
            "P_given_not_Q": 0.4,
            "Q": 0.15,
        },
        "checker": ExpectedEvidenceChecker(),
    },
}

for k, v in checkers.items():
    checker = v["checker"]
    # if k != "ee":
    #     continue
    answers = v["answers"]
    print("Checker:", k)
    time0 = time()
    result_plain = checker.max_min_arbitrage(answers, methods=('shgo',))
    time1 = time()
    print("No kwargs:", result_plain, "\nTime:", time1 - time0)
    # result_weighted = checker.max_min_arbitrage(answers, scoring=[0.5, 0.2])
    # print("Weighted:", result_weighted, "\nTime:", time() - time1)
    try:
        result_de = checker.de_method(answers, tmax=10, euler=False)
    except Exception as e:
        result_de = None
    time2 = time()
    print("DE:", result_de, "\nTime:", time2 - time1)
    try:
        result_de = checker.de_method(answers, tmax=1, euler=False)
    except Exception as e:
        result_de = None
    time3 = time()
    print("DE:", result_de, "\nTime:", time3 - time2)
    print("\n")



c:\Users\abhim\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


LOCAL_CACHE: None
max_concurrent_queries set for global semaphore: 25
Resetting global semaphore, max concurrent queries: 25

 NO_CACHE 


 NO_CACHE 


 NO_CACHE 


 NO_CACHE 

The setting for WRITE_VERIFICATION in fq_body_generator.py: True


c:\Users\abhim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_model_construction.py:54: UserWarning: `check_question_type` overrides an existing Pydantic `@field_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


Checker: para
No kwargs: ({'P': 0.6202041028697819, 'para_P': 0.6202040977296525}, 0.18402096102103532) 
Time: 0.011022090911865234
DE: ({'P': 0.6200992627568875, 'para_P': 0.6200992627568879}, 0.18368285811427293) 
Time: 0.004972696304321289
DE: ({'P': 0.6200992627568875, 'para_P': 0.6200992627568879}, 0.18368285811427293) 
Time: 0.0032050609588623047


Checker: neg
No kwargs: ({'P': 0.3797958855199512, 'not_P': 0.6202040889991406}, 0.18402096566427245) 
Time: 0.011984825134277344
DE: ({'P': 0.3817496244629985, 'not_P': 0.6182503755370016}, 0.1777107528654352) 
Time: 0.0009999275207519531
DE: ({'P': 0.3817496244629985, 'not_P': 0.6182503755370016}, 0.1777107528654352) 
Time: 0.0019969940185546875


Checker: cond
No kwargs: ({'P': 0.15049007026155137, 'Q_given_P': 0.6015323926754887, 'P_and_Q': 0.14918718961380573}, 0.00037907098548739526) 
Time: 0.061035871505737305
DE: ({'P': 0.17564173019844695, 'Q_given_P': 0.6666630386978009, 'P_and_Q': 0.11709384957623598}, 0.007351439938471538) 

c:\Users\abhim\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


No kwargs: ({'P': 0.4363630496557125, 'P_given_Q': 0.3009706035555113, 'P_given_not_Q': 0.4626877323758402, 'Q': 0.1449752718968944}, 0.015346055778096446) 
Time: 1.2458586692810059
DE: ({'P': 0.34177795054712845, 'Q': 0.2862104055727398, 'P_given_Q': 0.5620904201910312, 'P_given_not_Q': 0.25343858306152045}, -1.01143798520197) 
Time: 0.0179595947265625
DE: ({'P': 0.34177795054712845, 'Q': 0.2862104055727398, 'P_given_Q': 0.5620904201910312, 'P_given_not_Q': 0.25343858306152045}, -1.01143798520197) 
Time: 0.022006988525390625




In [17]:
B, C, D = 0.31, 0.46, 0.146
A = B * D + C * (1 - D)

ExpectedEvidenceChecker().arbitrage(
    answers={
        "P": 0.5,
        "P_given_Q": 0.25,
        "P_given_not_Q": 0.4,
        "Q": 0.15,
    },
    arbitrageur_answers={
        "P": 0.435,
        "P_given_Q": 0.293,
        "P_given_not_Q": 0.460,
        "Q": 0.145,
    },
    outcome={
        "P": True,
        "P_given_Q": True,
        "P_given_not_Q": None,
        "Q": True,
    }
)
# %%

-0.014451927854368218

In [22]:
ExpectedEvidenceChecker().arbitrage(
    answers={
        "P": 0.5,
        "P_given_Q": 0.25,
        "P_given_not_Q": 0.4,
        "Q": 0.15,
    },
    arbitrageur_answers={
        "P": 0.435,
        "P_given_Q": 0.295,
        "P_given_not_Q": 0.460,
        "Q": 0.145,
    },
    outcome={
        "P": True,
        "P_given_Q": True,
        "P_given_not_Q": None,
        "Q": True,
    },
)
# %%

-0.007649180531615674

In [ ]:
# %%

from forecasters.consistent_forecaster import ConsistentForecaster
from forecasters.basic_forecaster import BasicForecaster
from common.datatypes import ForecastingQuestion
from static_checks.Checker import *
from time import time

cf = ConsistentForecaster(
    BasicForecaster(),
    checks=[
        NegChecker(),
        AndOrChecker(),
        CondChecker(),
        ParaphraseChecker(),
    ],
)
fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

import asyncio

# x = cf.call(fq)
x = asyncio.run(cf.call_async(
    fq,
    bq_func_kwargs={"model": "gpt-4o-mini"},
    instantiation_kwargs={"model": "gpt-4o-mini"},
    model="gpt-4o-mini",
))
# x = asyncio.run(cf.call_async(fq))

In [ ]:
# %%

print(f"{x=}")

raise

In [ ]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.43}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.57}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
}

for k, v in checkers.items():
    checker = v["checker"]
    answers = v["answers"]
    if k != "cons":
        continue
    print("Checker:", k)
    time0 = time()
    result_shgo = checker.max_min_arbitrage(answers, methods=("shgo",))
    time1 = time()
    print("SHGO:", result_shgo, "\nTime:", time1 - time0)
    result_diff_evol = checker.max_min_arbitrage(
        answers, methods=("differential_evolution",)
    )
    time2 = time()
    print("Diff Evolution:", result_diff_evol, "\nTime:", time2 - time1)
    result_dual_annealing = checker.max_min_arbitrage(
        answers, methods=("dual_annealing",)
    )
    time3 = time()
    print("Dual Annealing:", result_dual_annealing, "\nTime:", time3 - time2)
    result_basinhopping = checker.max_min_arbitrage(answers, methods=("basinhopping",))
    time4 = time()
    print("Basinhopping:", result_basinhopping, "\nTime:", time4 - time3)
    result_plain = checker.max_min_arbitrage(answers)
    time5 = time()
    print("No kwargs:", result_plain, "\nTime:", time5 - time4)
    print("\n\n")


# print('----------------')
# print(CondChecker().min_arbitrage(ex, exa))
# print('----------------')
# print(CondChecker().arbitrage(outcome1, ex, exa))
# print(CondChecker().arbitrage(outcome2, ex, exa))
# print(CondChecker().arbitrage(outcome3, ex, exa))
# print('----------------')

In [ ]:
# %%

import json
from common.datatypes import (
    ForecastingQuestion_stripped,
    ForecastingQuestion,
    Prob_cot,
    Prob,
    PlainText,
)
from common.llm_utils import query_api_chat_sync, query_api_chat_sync_native
import os

fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

fqs = ForecastingQuestion_stripped(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
)

print(fqs.__str__())

In [ ]:
# %%

# os.environ["USE_OPENROUTER"] = "True"
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Summarize the question for the user.",
    },
    {"role": "user", "content": fq.__str__()},
]
# response = query_api_chat_sync(messages=messages, verbose=True, model="mistralai/mistral-large")